To improve prediction, I use the features in the same module for splitting variables and use principal components (maybe incorporate time??) for regressors!

In [1]:
library("party")
library(MASS)
library("glmertree")
library("caret")
source("SimData.r")
library("randomForest")
library("rpart")
library("fuzzyforest")
library("WGCNA")

Warning message:
"package 'party' was built under R version 3.6.1"Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Warning message:
"package 'strucchange' was built under R version 3.6.1"Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich
Warning message:
"package 'sandwich' was built under R version 3.6.1"Warning message:
"package 'glmertree' was built under R version 3.6.1"Loading required package: lme4
Warning message:
"package 'lme4' was built under R version 3.6.1"Loading required package: Matrix

Attaching package: 'lme4'

The following object is masked from 'package:modeltools':

    refit

Loading required package: partykit
Warning message:
"package 'partykit' was built under R version 3.6.1"Loading required package: libcoin
Warning 

# Data

In [2]:
# training data
n <- 300 # number of patients
T <-  5 # number of observations per patients

set.seed(100)

data <- as.data.frame(sim_3(n, T)) 
colnames(data)[401] <- "y"


# add patient/cluster information
for (i in 1:n){
    data$patient[(1+(i-1)*T):(i*T)] = rep(i,T)
}

#add PC
data$PC1 = (prcomp(data[,1:100], scale = TRUE)$x)[,1]
# data$PC1 = scale(data$PC1)/norm(scale(data$PC1),type="2")
data$PC2 = (prcomp(data[,101:200], scale = TRUE)$x)[,1]
data$PC3 = (prcomp(data[,201:300], scale = TRUE)$x)[,1]

data[1:6,]

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V396,V397,V398,V399,V400,y,patient,PC1,PC2,PC3
0.4041948,-0.1157621,-0.8282505,-0.2551767,0.7114005,0.09327738,0.4703178,-0.5625359,0.71595493,0.1898496,...,0.71453271,-0.5817907,0.31863009,0.1169713,0.88678481,-2.108422,1,-0.9721104,-1.8244420,-4.7440900
-0.5048507,-1.1607588,-0.9535845,-1.1433429,-0.3331617,-1.11665992,-0.4615737,-1.5207829,0.02850286,-0.3527653,...,-0.05631991,-1.6737887,0.91176834,-0.5207258,0.08539332,-10.907004,1,7.1642065,-0.8376986,-10.5236702
-0.8002010,-0.8082677,-1.0642596,-1.0390025,-0.5794711,-0.98738083,-0.5376431,-1.4163408,-0.22285559,-0.4874867,...,-0.22036390,-2.4876058,0.85579524,0.1344519,-0.31106741,-9.646296,1,7.2189515,-10.7372208,-10.1342376
-0.9136394,-1.3453939,-1.2412924,-0.5843280,-0.7852858,-1.10955238,-1.0380377,-1.3915048,-0.74088698,-0.3579343,...,-0.07577874,-3.0802759,0.76572763,-0.3152833,0.01155116,-3.301271,1,9.0966711,-5.9562909,0.3599288
-1.1862461,-0.8985310,-1.1362628,-0.5670641,-0.4920586,-0.39728046,-0.6302260,-0.6027598,-0.23438185,-0.3974606,...,0.87427111,-2.9496439,0.05246792,-0.7299419,-0.76754000,-4.642500,1,6.3373150,-4.2034960,2.0482581
0.7559472,2.1693698,1.1034058,1.9401066,0.6839483,1.12735727,1.5714702,1.5045449,1.51050327,1.6902715,...,-0.55237209,0.2431390,-1.33757722,-0.2405078,-0.62730903,21.973421,2,-16.2854556,1.3671918,11.1755676


In [3]:
# test data
n_test <- 100
T <-  5 # number of observations per patients
set.seed(101)
data_test <- as.data.frame(sim_3(n_test, T)) 
colnames(data_test)[401] <- "y"


# add patient/cluster information
for (i in 1:n_test){
    data_test$patient[(1+(i-1)*T):(i*T)] = rep(i,T)
}

# pca
# DANGEROUS: the PC can be +PC or -PC, so the prediction on test data shoule
# better not use PC of data_test! (the coefficient of linear model is fixed)
data_test$PC1 = -(prcomp(data_test[,1:100], scale = TRUE)$x)[,1]
# data_test$PC1 = scale(data_test$PC1)/norm(scale(data_test$PC1),type="2")
data_test$PC2 = (prcomp(data_test[,101:200], scale = TRUE)$x)[,1]
data_test$PC3 = (prcomp(data_test[,201:300], scale = TRUE)$x)[,1] 


data_test[1:6,]

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V396,V397,V398,V399,V400,y,patient,PC1,PC2,PC3
0.2060153,0.64024620,0.34032334,1.05586322,1.5860748,0.8222607,0.3228772,1.9814633,0.1915002,0.1206337,...,-0.11273431,0.61878986,1.1739663,0.3107692,0.21435946,12.804991,1,5.4577805,-5.707052,3.575336
1.0581125,1.83637294,1.28134711,1.95099335,2.1117273,1.5162299,1.1903985,2.6089574,1.1034408,0.7703308,...,-0.07525858,-0.32767601,0.4441376,0.3337583,0.43528673,31.115945,1,14.2953109,6.727536,4.694684
0.4365846,1.46430543,0.75723534,1.59393501,1.4047144,0.6625449,0.5281878,1.8311688,0.8361954,0.9370753,...,-0.09250902,0.16902591,0.9010316,0.1224454,0.57227788,17.608712,1,9.6293159,3.598046,11.616276
0.3370362,1.27335072,0.85899400,1.14445890,1.0956487,0.4145852,0.3415175,1.8317050,0.4938670,1.2542567,...,0.17486941,0.03718500,1.0193002,0.2050546,0.07434591,10.629618,1,7.5569242,5.540730,3.241353
-1.0935606,-0.04208021,-0.06375357,-0.02680852,-0.2886002,-0.6175841,-0.8728703,0.5218694,-0.7432135,0.2455381,...,0.08441171,0.09446398,0.4766140,-1.2353549,-0.25145304,-10.864407,1,-3.5806759,10.413707,7.551542
0.2791273,0.75162924,-0.30528207,-0.21112131,-0.2949749,-0.2916753,0.1386101,0.7237688,0.8251524,0.4313684,...,-0.86593112,2.28742993,0.6374255,0.4740288,1.31109763,7.806119,2,-0.3818383,3.195289,-9.555655


In [75]:
# see what is the proportion of explained sum of squares
# prcomp(data_test[,201:300], scale = TRUE)
# prcomp(data_test[,301:400], scale = TRUE)

In [4]:
# Methods for extracting names of splitting features used in a tree
# tree: a tree object; data: the train or test set
get_split_names = function(tree,data){
    # path: the string that contains all the node information
    paths <- pre:::list.rules(tree, removecomplements = FALSE)
    vnames = names(data)
    # the regex for a variable
    # tomatch = paste(paste(var,"<="),"|",paste(var,">"),sep="")
    # match to tomatch in path
    tmp = vnames[sapply(sapply(vnames, FUN = function(var) grep(paste(paste(var,"<="),"|",paste(var,">"),sep=""), paths)), length) > 0]
    return (tmp)
}

# Gtree

In [98]:
regress_var = paste("PC",c(1),sep="")
# regress_var = paste("PC",c(1,2,3),sep="")
# regress_var = paste("V",c(1,2,3,60,301,302,306),sep="") # abs correct regressor
# regress_var = paste("V",c(1,2,3,301,302),sep="") # abs correct regressor
# regress_var = paste("V",301:400,sep="") # abs correct regressor
# regress_var = "1"

split_var = paste("V",1:100,sep="") # always use all the features to split
# split_var = paste("V",c(1,2,3,301,302),sep="") # abs correct regressor
# split_var = paste("V",c(1,2,3,60,301,302,306),sep="")
# split_var = paste("V",c(3,2),sep="")

cluster_var = "patient"
Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                           "|",cluster_var,"|",
                         paste(split_var,collapse = "+")))

In [99]:
system.time({
    # you can change alpha = 0.2
    lmm_tree <- lmertree(Formula, data = data,alpha = 0.2,maxdepth=5) 
})
# mean square error on test data
mean((predict(lmm_tree,newdata=data_test,re.form=NA)-data_test$y)**2)

# the selected features
get_split_names(lmm_tree$tree,data)

# coef(lmm_tree)
# plot(lmm_tree,which="tree",main = "glmertree")

   user  system elapsed 
  13.43    0.09   14.36 

[1] 922.8058

[1] "V1"  "V2"  "V3"  "V60"

# PCA TEST

In [53]:
X = matrix(rep(1,12),3,4)
# X = data.frame(X)
X
A = X%*%t(X)
A

1,1,1,1
1,1,1,1
1,1,1,1


4,4,4
4,4,4
4,4,4


In [54]:
ev <- eigen(A)
ev

eigen() decomposition
$values
[1] 1.200000e+01 3.552714e-15 0.000000e+00

$vectors
           [,1]       [,2]       [,3]
[1,] -0.5773503  0.8164966  0.0000000
[2,] -0.5773503 -0.4082483 -0.7071068
[3,] -0.5773503 -0.4082483  0.7071068


In [58]:
ev$vector[,1] # first PC

[1] -0.5773503 -0.5773503 -0.5773503

In [66]:
svd(X)$u[,1] # or use this directly

[1] -0.5773503 -0.5773503 -0.5773503

In [51]:
# PCA but i don't get it
prcomp(t(X))

Standard deviations (1, .., p=3):
[1] 0 0 0

Rotation (n x k) = (3 x 3):
     PC1 PC2 PC3
[1,]   1   0   0
[2,]   0   1   0
[3,]   0   0   1